In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds


In [3]:
dataset, info = tfds.load(
    "tf_flowers",
    as_supervised=True,
    with_info=True
)

c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Dl Completed...: 100%|██████████| 1/1 [00:24<00:00, 24.76s/ url]


Dataset tf_flowers downloaded and prepared to C:\Users\User\tensorflow_datasets\tf_flowers\3.0.1. Subsequent calls will reuse this data.


In [4]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 1) 라이브러리 임포트 / Import Libraries\n",
    "**[KOR]** 이 셀에서는 전체 파이프라인에 필요한 라이브러리를 임포트합니다.\n",
    "**[ENG]** This cell imports all libraries required for the entire pipeline."
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "import tensorflow as tf\n",
    "import tensorflow_datasets as tfds\n",
    "\n",
    "# Hugging Face 관련 라이브러리 / Hugging Face libraries\n",
    "from transformers import (ViTImageProcessor, ViTForImageClassification,\n",
    "                          TFViTModel, create_optimizer, TrainingArguments, Trainer,\n",
    "                          DefaultDataCollator)\n",
    "\n",
    "# W&B / Weights & Biases\n",
    "import wandb\n",
    "from wandb.keras import WandbCallback\n",
    "\n",
    "# 기타 / Others\n",
    "import numpy as np\n",
    "import os\n",
    "from datasets import Dataset"
   ],
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 2) W&B 프로젝트 초기화 / Initialize W&B\n",
    "**[KOR]** 여기서는 W&B 프로젝트를 초기화합니다. 원하는 프로젝트 이름과 실험명을 지정할 수 있습니다.\n",
    "**[ENG]** Here, we initialize a W&B project. Customize the project name and run name as desired."
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "wandb.init(\n",
    "    project=\"ViT_vs_CNN_flowers\", \n",
    "    name=\"Initial_Run\", \n",
    "    reinit=True\n",
    ")"
   ],
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 3) 데이터셋 로드 및 전처리 / Load and Preprocess the Dataset\n",
    "**[KOR]** `tf_flowers` 데이터셋을 가져와서 (image, label) 쌍으로 분할하고, 224×224로 리사이즈 및 0~1 범위 정규화 후 배치 형태로 준비합니다.\n",
    "**[ENG]** Load the `tf_flowers` dataset, split it into (image, label) pairs, resize to 224×224, normalize to [0,1], and prepare it in batches."
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "# Load dataset\n",
    "dataset, info = tfds.load(\"tf_flowers\", as_supervised=True, with_info=True)\n",
    "num_classes = info.features['label'].num_classes\n",
    "total_examples = info.splits['train'].num_examples\n",
    "\n",
    "train_size = int(total_examples * 0.8)\n",
    "val_size = int(total_examples * 0.1)\n",
    "test_size = int(total_examples * 0.1)\n",
    "\n",
    "# Preprocessing function\n",
    "def format_image(image, label, img_size=224):\n",
    "    image = tf.image.resize(image, (img_size, img_size))\n",
    "    image = tf.cast(image, tf.float32) / 255.0\n",
    "    return image, label\n",
    "\n",
    "ds_all = dataset['train'].map(lambda x, y: format_image(x, y, 224))\n",
    "\n",
    "# Split dataset\n",
    "train_ds = ds_all.take(train_size)\n",
    "val_ds = ds_all.skip(train_size).take(val_size)\n",
    "test_ds = ds_all.skip(train_size + val_size).take(test_size)\n",
    "\n",
    "batch_size = 32\n",
    "train_batches = train_ds.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)\n",
    "val_batches = val_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)\n",
    "test_batches = test_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)\n",
    "\n",
    "print(f\"Number of classes: {num_classes}\")\n",
    "print(f\"Total samples: {total_examples}\")"
   ],
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 4) 간단 CNN 모델 정의 / Define a Simple CNN Model\n",
    "**[KOR]** 전통적인 CNN 모델과 비교하기 위해, 간단한 Conv-BN-MaxPool 구조의 CNN 모델을 정의합니다.\n",
    "**[ENG]** Define a simple CNN model (Conv-BN-MaxPool architecture) for comparison with the ViT model."
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "def create_simple_cnn(num_classes):\n",
    "    inputs = tf.keras.Input(shape=(224, 224, 3))\n",
    "    x = tf.keras.layers.Conv2D(32, 3, activation='relu')(inputs)\n",
    "    x = tf.keras.layers.MaxPool2D()(x)\n",
    "    x = tf.keras.layers.Conv2D(64, 3, activation='relu')(x)\n",
    "    x = tf.keras.layers.MaxPool2D()(x)\n",
    "    x = tf.keras.layers.Conv2D(128, 3, activation='relu')(x)\n",
    "    x = tf.keras.layers.GlobalAveragePooling2D()(x)\n",
    "    x = tf.keras.layers.Dense(256, activation='relu')(x)\n",
    "    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)\n",
    "    model = tf.keras.Model(inputs, outputs)\n",
    "    return model"
   ],
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 5) CNN 모델 훈련 / Train the CNN Model\n",
    "**[KOR]** 간단 CNN 모델을 학습시키고 W&B에 로깅합니다. CNN 성능을 파악한 뒤, 이후 ViT와 비교할 수 있습니다.\n",
    "**[ENG]** Train the simple CNN model and log metrics to W&B. We can then compare its performance to the ViT model."
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "# CNN model\n",
    "cnn_model = create_simple_cnn(num_classes)\n",
    "\n",
    "wandb.init(\n",
    "    project=\"ViT_vs_CNN_flowers\", \n",
    "    name=\"CNN_flowers_run\", \n",
    "    reinit=True\n",
    ")\n",
    "\n",
    "cnn_model.compile(\n",
    "    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),\n",
    "    loss='sparse_categorical_crossentropy',\n",
    "    metrics=['accuracy']\n",
    ")\n",
    "\n",
    "cnn_model.fit(\n",
    "    train_batches,\n",
    "    validation_data=val_batches,\n",
    "    epochs=3,  # 예시로 3 epoch\n",
    "    callbacks=[WandbCallback()]\n",
    ")\n",
    "\n",
    "# Evaluate on test set\n",
    "cnn_eval = cnn_model.evaluate(test_batches)\n",
    "print(\"CNN Test Evaluation:\", cnn_eval)\n",
    "wandb.finish()"
   ],
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 6) Functional API 기반 ViT 모델 빌드 함수 / Build ViT Model with Functional API\n",
    "**[KOR]** 사전 학습된 ViT를 불러와, Dense와 Dropout, BatchNorm 등을 파라미터로 동적으로 추가/조절하는 함수를 만듭니다.\n",
    "**[ENG]** Load a pretrained ViT, then add Dense, Dropout, and BatchNorm layers dynamically based on hyperparameters in a build function."
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "def build_vit_model(\n",
    "    num_classes,\n",
    "    num_dense_layers=1,\n",
    "    dense_units=128,\n",
    "    dropout_rate=0.3,\n",
    "    use_batchnorm=True,\n",
    "    base_trainable=False\n",
    "):\n",
    "    # Pretrained ViT (예: google/vit-base-patch16-224-in21k)\n",
    "    base_vit = TFViTModel.from_pretrained(\"google/vit-base-patch16-224-in21k\")\n",
    "    base_vit.trainable = base_trainable\n",
    "\n",
    "    inputs = tf.keras.Input(shape=(224, 224, 3), name=\"input_images\")\n",
    "    vit_outputs = base_vit(inputs).last_hidden_state  # [batch, seq_len, hidden_dim]\n",
    "\n",
    "    # 시퀀스 차원 풀링 / Sequence dimension pooling\n",
    "    x = tf.keras.layers.GlobalAveragePooling1D()(vit_outputs)\n",
    "\n",
    "    # 동적으로 Dense 레이어 쌓기 / Dynamically stack Dense layers\n",
    "    for _ in range(num_dense_layers):\n",
    "        x = tf.keras.layers.Dense(dense_units, activation='relu')(x)\n",
    "        if use_batchnorm:\n",
    "            x = tf.keras.layers.BatchNormalization()(x)\n",
    "        x = tf.keras.layers.Dropout(dropout_rate)(x)\n",
    "\n",
    "    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)\n",
    "    model = tf.keras.Model(inputs, outputs)\n",
    "    return model"
   ],
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 7) ViT 모델 학습 (단일 실험 예시) / Train the ViT Model (Single Experiment Example)\n",
    "**[KOR]** 위에서 정의한 `build_vit_model`을 사용해 ViT 모델을 하나 구성해 보고, W&B에 로깅합니다.\n",
    "**[ENG]** Use `build_vit_model` to create a ViT model, then train and log metrics to W&B as a single-run example."
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "# Example single-run training\n",
    "wandb.init(\n",
    "    project=\"ViT_vs_CNN_flowers\",\n",
    "    name=\"ViT_single_run\",\n",
    "    reinit=True\n",
    ")\n",
    "\n",
    "# Build a ViT model with some chosen hyperparameters\n",
    "vit_model = build_vit_model(\n",
    "    num_classes=num_classes,\n",
    "    num_dense_layers=2,\n",
    "    dense_units=128,\n",
    "    dropout_rate=0.3,\n",
    "    use_batchnorm=True,\n",
    "    base_trainable=False\n",
    ")\n",
    "\n",
    "vit_model.compile(\n",
    "    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),\n",
    "    loss='sparse_categorical_crossentropy',\n",
    "    metrics=['accuracy']\n",
    ")\n",
    "\n",
    "vit_model.fit(\n",
    "    train_batches,\n",
    "    validation_data=val_batches,\n",
    "    epochs=3,\n",
    "    callbacks=[WandbCallback()]\n",
    ")\n",
    "\n",
    "# Evaluate on test set\n",
    "test_loss, test_acc = vit_model.evaluate(test_batches)\n",
    "wandb.log({\"test_loss\": test_loss, \"test_accuracy\": test_acc})\n",
    "print(\"ViT Test Accuracy:\", test_acc)\n",
    "wandb.finish()"
   ],
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 8) W&B Sweep 설정 예시 (sweep.yaml) / W&B Sweep Configuration Example\n",
    "**[KOR]** 다양한 하이퍼파라미터(레이어 수, Dropout 비율, BatchNorm 여부 등)를 탐색하기 위한 예시 스윕 설정입니다.\n",
    "**[ENG]** An example sweep configuration for exploring various hyperparameters (number of Dense layers, dropout rate, batch normalization, etc.)."
   ]
  },
  {
   "cell_type": "code",
   "metadata": {
    "collapsed": true
   },
   "source": [
    "sweep_config = '''\n",
    "method: bayes\n",
    "metric:\n",
    "  name: val_accuracy\n",
    "  goal: maximize\n",
    "parameters:\n",
    "  num_dense_layers:\n",
    "    values: [1, 2, 3]\n",
    "  dense_units:\n",
    "    values: [64, 128, 256]\n",
    "  dropout_rate:\n",
    "    values: [0.2, 0.3, 0.5]\n",
    "  use_batchnorm:\n",
    "    values: [true, false]\n",
    "  base_trainable:\n",
    "    values: [false, true]\n",
    "  learning_rate:\n",
    "    values: [1e-4, 1e-5]\n",
    "  batch_size:\n",
    "    values: [16, 32]\n",
    "  epochs:\n",
    "    values: [5, 10]\n",
    "'''\n",
    "\n",
    "print(sweep_config)"
   ],
   "execution_count": null,
   "outputs": [
    {
     "output_type": "stream",
     "text": [
      "method: bayes\n",
      "metric:\n",
      "  name: val_accuracy\n",
      "  goal: maximize\n",
      "parameters:\n",
      "  num_dense_layers:\n",
      "    values: [1, 2, 3]\n",
      "  dense_units:\n",
      "    values: [64, 128, 256]\n",
      "  dropout_rate:\n",
      "    values: [0.2, 0.3, 0.5]\n",
      "  use_batchnorm:\n",
      "    values: [true, false]\n",
      "  base_trainable:\n",
      "    values: [false, true]\n",
      "  learning_rate:\n",
      "    values: [1e-4, 1e-5]\n",
      "  batch_size:\n",
      "    values: [16, 32]\n",
      "  epochs:\n",
      "    values: [5, 10]\n"
     ],
     "name": "stdout"
    }
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 9) Sweep 트레이닝 함수 / Sweep Training Function\n",
    "**[KOR]** Sweep 에이전트가 각 실험마다 호출할 함수입니다. 여러 조합을 자동 시도하며 결과를 W&B에 기록합니다.\n",
    "**[ENG]** This function is called by the sweep agent for each experiment, automatically trying different combinations and logging results to W&B."
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "def train_sweep(config=None):\n",
    "    with wandb.init(config=config):\n",
    "        config = wandb.config\n",
    "\n",
    "        # 데이터셋의 배치를 다시 설정하려면 아래와 같이 batch_size 재지정 가능\n",
    "        # If you want to re-batch data based on config.batch_size, do:\n",
    "        current_train_batches = train_ds.batch(config.batch_size).prefetch(tf.data.AUTOTUNE)\n",
    "        current_val_batches = val_ds.batch(config.batch_size).prefetch(tf.data.AUTOTUNE)\n",
    "        current_test_batches = test_ds.batch(config.batch_size).prefetch(tf.data.AUTOTUNE)\n",
    "\n",
    "        # 모델 빌드\n",
    "        model = build_vit_model(\n",
    "            num_classes=num_classes,\n",
    "            num_dense_layers=config.num_dense_layers,\n",
    "            dense_units=config.dense_units,\n",
    "            dropout_rate=config.dropout_rate,\n",
    "            use_batchnorm=config.use_batchnorm,\n",
    "            base_trainable=config.base_trainable\n",
    "        )\n",
    "\n",
    "        optimizer = tf.keras.optimizers.Adam(learning_rate=config.learning_rate)\n",
    "        model.compile(\n",
    "            optimizer=optimizer,\n",
    "            loss='sparse_categorical_crossentropy',\n",
    "            metrics=['accuracy']\n",
    "        )\n",
    "\n",
    "        model.fit(\n",
    "            current_train_batches,\n",
    "            validation_data=current_val_batches,\n",
    "            epochs=config.epochs,\n",
    "            callbacks=[WandbCallback()]\n",
    "        )\n",
    "\n",
    "        test_loss, test_acc = model.evaluate(current_test_batches)\n",
    "        wandb.log({\"test_loss\": test_loss, \"test_accuracy\": test_acc})\n",
    "        print(\"Test Accuracy:\", test_acc)"
   ],
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 10) Sweep 실행 / Launch the Sweep\n",
    "**[KOR]** 아래 코드(또는 CLI 명령어)를 통해 sweep을 생성하고, 에이전트를 실행하면 다양한 아키텍처 및 HP 조합이 자동으로 탐색됩니다.\n",
    "**[ENG]** Use the code (or CLI commands) below to create a sweep and run an agent, automatically exploring multiple architecture and hyperparameter combinations."
   ]
  },
  {
   "cell_type": "code",
   "metadata": {},
   "source": [
    "# (주의) 실제로 실행할 때 주석 해제\n",
    "# import yaml\n",
    "# sweep_config_dict = yaml.safe_load(sweep_config)\n",
    "# sweep_id = wandb.sweep(sweep_config_dict, project=\"ViT_vs_CNN_flowers\")\n",
    "# wandb.agent(sweep_id, function=train_sweep)"
   ],
   "execution_count": null,
   "outputs": []
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 요약 / Summary\n",
    "**[KOR]** 위 노트북을 통해 전통적인 CNN과 사전 학습된 ViT 기반 모델을 모두 학습해보고, W&B의 Sweep 기능으로 아키텍처 및 하이퍼파라미터를 자동 탐색할 수 있습니다.\n",
    "**[ENG]** This notebook allows you to train both a traditional CNN and a pretrained ViT model, and use W&B Sweeps to automatically search for optimal architectures and hyperparameters."
   ]
  }
 ],
 "metadata": {
  "language_info": {
   "name": "python"
  },
  "colab": {
   "name": "vit_cnn_flowers_sweeps.ipynb"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}


NameError: name 'null' is not defined